# Imports

In [11]:
import boto3
from boto3.dynamodb.conditions import Key
import requests
from requests.auth import HTTPBasicAuth
import json
import decimal



# Initialization

In [12]:
# ---- Config ----
AWS_PROFILE = "musa"  
TABLE_NAME = "TargetsAccounts2"
PARTITION_KEY = "SupplierID"
PARTITION_VALUE = "DWS"

API_URL = "https://rapi.rapidtradews.com/post/account"
API_USER = "DWS"
API_PASS = "METUNG"

# Functions

In [13]:
def convert_decimal(obj):
    if isinstance(obj, list):
        return [convert_decimal(i) for i in obj]
    elif isinstance(obj, dict):
        return {k: convert_decimal(v) for k, v in obj.items()}
    elif isinstance(obj, decimal.Decimal):
        # Convert to int if possible, else float
        if obj % 1 == 0:
            return int(obj)
        else:
            return float(obj)
    else:
        return obj

def fetch_all_items():
    """Fetch all items from DynamoDB with SupplierID = DWS (with pagination)."""
    session = boto3.Session(profile_name=AWS_PROFILE)
    dynamodb = session.resource("dynamodb")
    table = dynamodb.Table(TABLE_NAME)

    items = []
    last_evaluated_key = None
    total_fetched = 0  # Track how many accounts fetched

    while True:
        if last_evaluated_key:
            response = table.query(
                KeyConditionExpression=Key(PARTITION_KEY).eq(PARTITION_VALUE),
                ExclusiveStartKey=last_evaluated_key
            )
        else:
            response = table.query(
                KeyConditionExpression=Key(PARTITION_KEY).eq(PARTITION_VALUE)
            )

        batch_items = response.get("Items", [])
        items.extend(batch_items)
        total_fetched += len(batch_items)
        print(f"Fetched {total_fetched} accounts so far...")  # Progress update

        last_evaluated_key = response.get("LastEvaluatedKey")
        if not last_evaluated_key:
            break

    print(f"Total accounts fetched: {total_fetched}")  # Final count
    return items  # <-- Add this line

def post_to_api(data):
    """Post JSON array to API with basic auth."""
    headers = {"Content-Type": "application/json"}
    data = convert_decimal(data)  # Convert Decimals before dumping
    response = requests.post(
        API_URL,
        data=json.dumps(data),
        headers=headers,
        auth=HTTPBasicAuth(API_USER, API_PASS),
        timeout=30
    )
    return response

# Main Handler

In [14]:
if __name__ == "__main__":
    # Step 1: Fetch all items
    print("Fetching items from DynamoDB...")
    items = fetch_all_items()
    print(f"Fetched {len(items)} items")

    # Step 2: Post to API
    print("Posting data to API...")
    response = post_to_api(items)

    print("Status:", response.status_code)
    print("Response:", response.text)

Fetching items from DynamoDB...
Fetched 1066 accounts so far...
Fetched 2151 accounts so far...
Fetched 3243 accounts so far...
Fetched 4340 accounts so far...
Fetched 5437 accounts so far...
Fetched 6539 accounts so far...
Fetched 7630 accounts so far...
Fetched 8706 accounts so far...
Fetched 9776 accounts so far...
Fetched 10215 accounts so far...
Total accounts fetched: 10215
Fetched 10215 items
Posting data to API...
Status: 200
Response: 
